In [1]:
from manager import load_obj
import os
import re
import string
from collections import Counter
from pprint import pprint
from tqdm.notebook import tqdm
from itertools import combinations

In [2]:
works = load_obj("cleaned_corpus")

In [3]:
count= 0
for work in works:
    if work.references != "none":
        count += 1

In [4]:
count

502

# Create a list of all parsed names from the exparser occuring in the bibliography

In [5]:
index_work = {}
for index,work in enumerate(works):
    index_work[work.link.split("/")[-1]] = index

In [6]:


def find_tag(text, tag):
    """
        Arguments:
            text: a string from the xml
            tag: a string which tag should be search for
        Returns:
            The content between two tags
    """
    reg_start = re.compile(fr"(\<{tag}\>)")
    reg_end =  re.compile(fr"(\<\/{tag}\>)")
    start = [m.start() for m in re.finditer(reg_start, text)]
    end = [m.start() for m in re.finditer(reg_end, text)]
    
    aut=""
    if start and end:
        for s,e in zip(start,end):
            aut += re.sub(r"<[^>]*>", "", text[s:e] + "  ")
    else:
        aut = ""
    return aut

In [7]:
def create_ref_dic(text):
    """-creates a dictionary with all authors from a bibliography
       -gets as input the parsed authors from EXparser 
    """
    references ={"autoren": []}
    refs = text.split("<\/sequence")
    for ref in refs:
        text = find_tag(ref, "author")
        authors = re.split("  |\/", text)
        
        authors = [x for x in authors if len(x) > 0]
        if len(authors) != 0:
            references["autoren"] += authors
    return references

In [8]:
path = r"C:\Users\Tim\Desktop\bachelorarbeit\Parsed_References" #needs to be edited
for filename in os.listdir(r"C:\Users\Tim\Desktop\bachelorarbeit\Parsed_References"):
    with open(os.path.join(path, filename), 'r', encoding = "utf-8") as f: # open in readonly mode
        name = os.path.basename(f.name).split(".")[0]
        if index_work.get(name) != None:
            works[index_work.get(name)].ref_authors = create_ref_dic(f.read())

In [9]:
for work in works:
    if work.link == "https://www.hausarbeiten.de/document/359235":
        print(work.ref_authors)

{'autoren': []}


In [10]:
namelist={}
for work in works:
    names =[]
    for name in work.ref_authors["autoren"]:
        if len(name.split(" ")) == 2 and (name.split(" ")[0] == "" or name.split(" ")[1] == "") :
            continue
        elif len(name.split(" ")) > 1 and len(name.split(" ")) < 5:
            names.append(name.lower())
        else:
            continue
    namelist[work.link] = names

In [11]:
namelist

{'https://www.hausarbeiten.de/document/889420': ['barner, wilfried',
  'brake, michael, lars',
  'glanz, berit',
  'glanz, berit',
  'glaser, peter',
  'glaser, peter',
  'hertwig, johannes',
  'schlegel, friedrich',
  'schlegel, friedrich',
  'schmitz-emans, monika. „der'],
 'https://www.hausarbeiten.de/document/1180614': ['schiller, friedrich',
  'schiller, friedrich',
  'alt, peter-andr',
  'luserke-jaqui, matthias',
  'luserke-jaqui, matthias',
  'middel, carina',
  'nilges, yvonne',
  'schmidhäuser, eberhard',
  'borgards, roland',
  'martus, steffen',
  'vogl, joseph',
  'mazza, ethel',
  'willems, gottfried'],
 'https://www.hausarbeiten.de/document/1035784': ['lovecraft, h.p',
  'lovecraft, h.p',
  'lovecraft, h.p',
  'lovecraft, h.p',
  'lovecraft, h.p',
  'lovecraft, h.p.: vom',
  'stoker, bram',
  'amann, wilhelm',
  'frenschkowski, marco',
  'freyermuth, gundolf s.',
  'glaser, frank',
  'günzel, stephan',
  'hanke, christiane',
  'jones, mark',
  'kreienbrink, mattias',
  '

In [13]:
"""
creates a csv file, where the first Element is the URL of a work, as a identifier, and the following elements 
are the names occuring in the bibliography of the specific work

"""
import csv 

with open("names_with_work.txt", "w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f, delimiter = "\t")
    for key in namelist:
        output = [key, *namelist[key]]
        writer.writerow(output)
        